In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# matplotlib 한글 꺠짐 방지
# apt-get update
# apt-get install fonts-nanum* 
# apt-get install fontconfig
# fc-cache -fv  # font 캐시 날리기
# rm -rf /home/hy1/.cache/matplotlib/  #matplotliob 폰트 캐시 날리기
# !pip install matplotlib

In [3]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [4]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [5]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

## Transform

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수의 상관관계를 구해보자

In [6]:
##면적 인구정보
area = spark.read.csv('/corona_data/loc/sido_area.csv', encoding='CP949', header=True)
popu = spark.read.csv('/corona_data/loc/sido_population.csv', encoding='CP949', header=True)

area.show(3)
popu.show(3)

NameError: name 'spark' is not defined

In [7]:
sc

''

In [9]:
area_popu = area.join(popu, on='loc')
#area_popu.show()
area_popu = area_popu.select(area_popu.loc, ceil((area_popu.total/area_popu.area)).alias('popu_density'))
area_popu.show()

NameError: name 'area' is not defined

In [10]:
# 지역명과, 10만명 당 감염 현황
tmp2 = tmp.select('items').first()
co_patient = spark.createDataFrame(tmp2['items'])
#co_patient.show(3)

# tmp3 = co_patient.select(co_patient.gubun.alias('loc'), co_patient.qurRate)
# tmp3.show(3)
# co_rate = area_popu.join(tmp3, on='loc').distinct().orderBy(col('loc'))
# co_rate.show()

In [85]:
# 시각화
pd_co = co_rate.toPandas()
pd_co['qurRate'] = pd_co['qurRate'].astype(int)
pd_co.plot(kind='scatter'
           ,x='popu_density'
           ,y='qurRate'
           ,ylim=(30000,50000)
           ,figsize=(6, 6)
          )

+----+------------+-------+
| loc|popu_density|qurRate|
+----+------------+-------+
|강원|          93|  39566|
|경기|        1366|  41938|
|경남|         321|  37690|
|경북|         141|  34789|
|광주|        2920|  42137|
|대구|        2733|  36677|
|대전|        2722|  41017|
|부산|        4411|  37579|
|서울|       16093|  44168|
|세종|         811|  43412|
|울산|        1072|  40562|
|인천|        2831|  41435|
|전남|         152|  36958|
|전북|         226|  38925|
|제주|         378|  42605|
|충남|         265|  39130|
|충북|         221|  40338|
+----+------------+-------+



### 2.  4차 예방접종 완료자와 코로나 확진자 수의 상관관계를 구해보자

In [88]:
file_name = '/corona_data/vaccine/corona_vaccine_' + cal_std_day(1) + '.json'
vaccine = spark.read.json(file_name)
#vaccine.show(3)
vaccine_df = spark.createDataFrame(vaccine.select('data').first()['data'])
#vaccine_df.show()

pop_vaccine = vaccine_df.join(popu, on='loc').select('loc', col('v4'), col('total'),
                                                     ceil(
                                                         (col('v4')/col('total')) * 100
                                                     ).alias('v_rate'))
#pop_vaccine.show(100)
tmp3 = co_patient.select(co_patient.gubun.alias('loc'), co_patient.qurRate)
co_vaccine_df = pop_vaccine.join(tmp3, on='loc')
co_vaccine_df = co_vaccine_df.select('loc', 'v_rate', 'qurRate').distinct()
co_vaccine_df.show(3)

+----+------+-------+
| loc|v_rate|qurRate|
+----+------+-------+
|서울|    11|  44168|
|부산|    13|  37579|
|대구|    10|  36677|
|인천|    12|  41435|
|광주|    15|  42137|
|대전|    12|  41017|
|울산|    10|  40562|
|세종|     9|  43412|
|경기|    11|  41938|
|강원|    16|  39566|
|충북|    16|  40338|
|충남|    15|  39130|
|전북|    19|  38925|
|전남|    22|  36958|
|경북|    13|  34789|
|경남|    13|  37690|
|제주|    12|  42605|
+----+------+-------+



In [ ]:
pd_vac = co_vaccine_df.toPandas()

pd_vac['qurRate'] = pd_vac['qurRate'].astype(int)
pd_vac.plot(kind='scatter'
           ,x='v_rate'
           ,y='qurRate'
           ,figsize=(7, 7)
          )

### 3. 다중이용시설과 코로나 확진자 수의 상관관계

In [90]:

facility = spark.read.csv('/corona_data/loc/전국다중이용시설.csv', encoding='CP949', header=True)
facility.show(3)

In [92]:
fac_cnt = facility.groupBy(facility.광역.alias('loc')).agg(count('*').alias('fac_cnt'))
tmp = popu.join(fac_cnt, on='loc')
fac_popu = tmp.select('loc', ceil(tmp.fac_cnt/tmp.total*100000).alias('fac_popu'))

tmp3 = co_patient.select(co_patient.gubun.alias('loc'), co_patient.qurRate)
fac_rate = fac_popu.join(tmp3, on='loc').distinct()
fac_rate.show()

MapPartitionsRDD[924] at javaToPython at <unknown>:0


+----+--------+-------+
| loc|fac_popu|qurRate|
+----+--------+-------+
|경북|      36|  34789|
|대전|      53|  41017|
|전북|      42|  38925|
|충북|      38|  40338|
|울산|      38|  40562|
|경남|      41|  37690|
|제주|      45|  42605|
|충남|      38|  39130|
|전남|      45|  36958|
|인천|      46|  41435|
|부산|      44|  37579|
|대구|      43|  36677|
|경기|      46|  41938|
|서울|      54|  44168|
|광주|      51|  42137|
|세종|      65|  43412|
|강원|      38|  39566|
+----+--------+-------+



In [ ]:
pd_fac = fac_rate.toPandas()

pd_fac['qurRate'] = pd_fac['qurRate'].astype(int)
pd_fac.plot(kind='scatter'
           ,x='fac_popu'
           ,y='qurRate'
           ,ylim=(40000,55000)
           ,figsize=(6, 6) 
          )

### 4. 요일별 코로나 확진자 수를 구해보자

In [110]:
file_name = '/corona_data/patient/'
tmp = spark.read.json(file_name, encoding='UTF-8')

In [138]:
data = []
for r1 in tmp.select(tmp.items).toLocalIterator():
    if not r1.items:
        continue
    for r2 in r1.items:
        data.append(r2)

patient_data = spark.createDataFrame(data)
p = patient_data.withColumn('day_of_week', dayofweek(col('stdDay')))

# localOccCnt : 당일 발생한 코로나확진자 수 - 해외유입수
# 요일별 발생한 환자 수의 합을 구한 뒤 요일로 오름차순 정렬하시오
p = p.groupBy(p.day_of_week).agg(sum(col('localOccCnt')).alias('patients')).sort(col('day_of_week'))
p.show()

+-----------+---------+
|day_of_week| patients|
+-----------+---------+
|          1|3730348.0|
|          2|2381642.0|
|          3|5038466.0|
|          4|6983162.0|
|          5|5646140.0|
|          6|4312480.0|
|          7|4161302.0|
+-----------+---------+



In [ ]:
pd_week = p.toPandas()
pd_week.plot(kind='bar', figsize=(6, 6))